<a href="https://colab.research.google.com/github/manuvillalba-uclm/Natural_Lenguaje_Processing-ELM/blob/Preprocessing-optional/Machine_Learning_NLP_PreProcesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TO DO


Los emojis los traduce bien pero no como queremos, nos da igual que nos diga face_with_tears_of_joy, el objetivo es que tears y joy se integren con el texto, eso hay que revisarlo.

No sé hasta que punto nos interesa mantener los hastag




In [1]:
# dataframes library
!pip install pyspellchecker 
!pip install contractions
!pip install nltk --upgrade
!pip install emoji

import pandas as pd
import re
import nltk
nltk.download("popular")
nltk.download('vader_lexicon') #sentiment analysis
nltk.download('twython') #twitter not necessary

from nltk.tokenize.casual import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

#Correcciones y emoji
import emoji
from textblob import TextBlob
from spellchecker import SpellChecker
import contractions
from multiprocessing import Pool

#Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

#Feature selection 
from sklearn.feature_selection import SelectKBest, chi2

#Classification
from sklearn.model_selection import train_test_split

from sklearn import neighbors
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels



     |████████████████████████████████| 1.9MB 5.5MB/s 
     |████████████████████████████████| 317kB 5.9MB/s 
     |████████████████████████████████| 245kB 27.4MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81700 sha256=d8b91741d0563732db97350162a2025b535acccc2cca13ea4bd167e914c69bf0
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
     |████████████████████████████████| 1.4MB 5.5MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434675 sha256=59824694703e8618e3dabf00a3de44472cac3441cd569a166b01388f90e5a0f6
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 51kB 2.4MB/s 
  Created wheel for

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [2]:
english_stops = set(stopwords.words('english'))
spell = SpellChecker()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()


In [3]:
raw_dataset_tweets = pd.read_csv('https://raw.githubusercontent.com/manuvillalba-uclm/Natural_Lenguaje_Processing-ELM/Preprocessing-optional/labeled_data.csv?token=AGI3DY5GNUGRFLUE4RKP4L273PNZE', index_col=[0])

# Preprocesing:

Mandatory preprocessing steps
 - Remove unseful data: ! "_ $% & / ( ) = _ ˆ* ¡@
 - Remove all capital letters
 - Correct wrong words: https://norvig.com/spell-correct.html
 - Lemmatize all terms

Optional preprocessing steps:
 - Remove contractions: don’t -> do not
 - Remove repeated words: great great show
 - Replace emoticons, for example,🙂 with “smile”, 😉with “ok”


We just need index and tweet content for this first part, so we drop Crowdflowers users information from the dataset.

In [4]:
dataset_tweets_preprocesing = raw_dataset_tweets.copy()
dataset_tweets_preprocesing.drop("count", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("hate_speech", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("offensive_language", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("neither", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("class", axis = 1, inplace = True)

In [5]:
dataset_tweets_preprocesing

,tweet
0,!!! RT @mayasolovely: As a woman you shouldn't...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...
25291,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
25292,"you've gone and broke the wrong heart baby, an..."
25294,young buck wanna eat!!.. dat nigguh like I ain...
25295,youu got wild bitches tellin you lies


Seleccionamos sólo unos pocos para trabajar al principio más fácilmente

In [6]:
#@title Texto de título predeterminado
num_tweets = 280 #@param {type:"slider", min:20, max:24783, step:10}

examples_preprocesing = dataset_tweets_preprocesing.iloc[0:num_tweets,0]
examples_preprocesing.reset_index(inplace = True, drop= True)
print(examples_preprocesing)

lista_tweets = []



0      !!! RT @mayasolovely: As a woman you shouldn't...
1      !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2      !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3      !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4      !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
                             ...                        
275    "@WBeezy_: "@QuaTheGreat: Aww their avis look ...
276    "@WBeezy_: "@Run_DRC: Its simple, dont be a ho...
277    "@WannabeKanye: whys this bitch boosting herse...
278    "@WitHaPrettyface: @_1nonlykeisha bet not be n...
279    "@X_XPDOTJDOT: Damn near gotta protect my twee...
Name: tweet, Length: 280, dtype: object


### Definimos las funciones que procesarán los tweets


In [25]:
def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in english_stops])

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def demojizar(arr):
    text_arr = []
    trozos_emoji = []
    for token in arr:
      wordemoji = emoji.demojize(token, delimiters=("", ""))
      trozos_emoji = wordemoji.split('_')
      for word in trozos_emoji:
        text_arr.append(word)
    return text_arr

def textConditions(token):
  if re.match("[`!\"_$%&/()=_ˆ*¡,@:.]+",token): #Signos de puntuacion
    return False
  if re.match("[-><~]+",token): #Signos de puntuacion
    return False
  if re.match("[^\u0000-\u007F]+",token): #Unicodes
    return False
  return True

def processArray(arr):
  text_arr = []
  list(set(arr))
  for token in arr:
    if textConditions(token):
      token = stemmer.stem(token)
      token = lemmatizer.lemmatize(token) # < ---- LEMMA
      text_arr.append(token)
  # try: #Aqui iria el tagger de palabras, pero solo me lo hace para letras (ERROR)
  #   arr_tags = nltk.pos_tag(text_arr)
  # except:
  #   arr_tags = text_arr
  return text_arr

In [32]:
%%time

ds_alt_twitter = examples_preprocesing.copy()
ds_alt_twitter = ds_alt_twitter.str.lower() #A minusc
ds_alt_twitter = ds_alt_twitter.apply(contractions.fix) #Arreglar contracciones
ds_alt_twitter = ds_alt_twitter.apply(contractions.fix) #Plural a singular

ds_alt_twitter = ds_alt_twitter.str.replace('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z0-9_]+[A-Za-z0-9-_]+)','') #Remover usuario
ds_alt_twitter = ds_alt_twitter.str.replace('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z0-9_]+[A-Za-z0-9-_]+)','') #Remover usuario
ds_alt_twitter = ds_alt_twitter.str.replace('rt','') #Remover rt

ds_alt_twitter = ds_alt_twitter.apply(stopwords) #Quitar Stopwords
ds_alt_twitter = ds_alt_twitter.apply(remove_urls) #Quitar url

ds_alt_twitter = ds_alt_twitter.apply(lambda x: str(TextBlob(x).correct()))
ds_alt_twitter = ds_alt_twitter.apply(TweetTokenizer().tokenize) #Tokenizar
ds_alt_twitter = ds_alt_twitter.apply(demojizar) #Traducir emoji a texto
ds_alt_twitter = ds_alt_twitter.apply(processArray) #Eliminar repetidas, Lemma y stem



['!']
['!']
['!']
[':']
['woman']
['complain']
['cleaning']
['house']
['.']
['&']
['man']
['always']
['take']
['trash']
['out']
['...']
['!']
['!']
['!']
[':']
['boy']
['days']
['cold']
['...']
['tea']
['down']
['bad']
['coffin']
['dat']
['he']
['st']
['place']
['!']
['!']
['!']
['!']
['!']
['dawn']
['!']
['!']
['!']
[':']
['ever']
['fuck']
['bitch']
['sta']
['cry']
['?']
['confused']
['shit']
['!']
['!']
['!']
[':']
['look']
['like']
['tyranny']
['!']
['!']
['!']
[':']
['shit']
['hear']
['might']
['true']
['might']
['baker']
['bitch']
['told']
['a']
['\ue011']
['!']
['!']
['!']
['"']
[':']
['shit']
['blows']
['me']
['..']
['claim']
['faithful']
['somebody']
['still']
['fucking']
['hoes']
['!']
['face', 'with', 'tears', 'of', 'joy']
['face', 'with', 'tears', 'of', 'joy']
['face', 'with', 'tears', 'of', 'joy']
['"']
['!']
['!']
['!']
['"']
[':']
['sit']
['hate']
['another']
['bitch']
['..']
['got']
['much']
['shit']
['going']
['on']
['!']
['"']
['!']
['!']
['!']
['“']
[':']
['I']
['tire

In [27]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

ds_alt_twitter.to_csv(r'/content/gdrive/My Drive/preprocesado.csv')

Mounted at /content/gdrive
